# Phage host prediction

***

## Introduction

While the identification of phage has rapidly increased in the metagenomics age, matching identified phage to their range of host organisms remains a significant limitation of the field. For phage identified in cultured prokaryote isolates or known to infect specific eukaryotic cells, the pairing of phage to host is immediately known (although, the full *range* of posssible hosts may still require further study). In contrast, in the jumbled sea of assembled contigs in a metagenomics study, identifying specific pairings between host (such as prokaryote) genomes and individual viral genomes is a much more complex task. 

Understanding the host, or range of hosts, that a virus is capable of infecting is vital to understanding viral effects on individual organisms in an ecosystem as well as the dynamics of the ecosystem as a whole. There is also revitalised interest in the use of phage therapy to target "problematic" microbes (such as "pathogenic" bacteria associated with disease) as an alternative to antibiotics. And while phage discovery is in rapid growth, accurately understanding the range of organisms that many of these phage can infect, and their potential direct effect on those organisms, is currently lagging behind.

Several tools and methods have been developed that attempt to address this limitation in viral metagenomics, with the aim of predicting matches between viral genomes and the likely target host that they infect. Some of these include: 

- machine learning-based approaches *RaFAH*, *HostG*, and *VirHostMatcher*
- tRNA pairwise *blast* matching between identified viral tRNA and prokaryote metagenome-assembled genome (MAG) tRNA sequences
- nucleotide sequence identity based on pairwise *blast* between viral genomes and full prokaryote MAGs
- *blast* matching of CRISPR spacers identified in viral genomes against prokaryote MAGs
- identifying integrated prophage genomes contained within larger contigs (including stretches of host genome) that were also binned into specific prokaryote MAGs

In practice, several of these approaches (such as the machine learning-based tools) are limited by the fact that reference databases on which they're developed currently still include a small fraction of the likely full viral diversity. We have found that several of the other approaches (such as tRNA blast matching and nucleotide sequence identity) can provide an indication of likely match, but are not difinitive. CRSIPR matching is currently the most robust approach, but in our experience only returns a limited number of hits. Finally, prophage identified directly in binned MAGs have the caveat that differing GC content and coverage (particularly if the virus is replicating at the time) of the viral region within the contig *may* have affected the bin placement, and this contig may not actually correctly belong to the prokaryote MAG that it has been assigned to. Furthermore, each of the latter approaches (CRISPR; tRNA; genome sequence identity) require having available relevant prokaryote references (such as a set of MAGs from the same system (although, in practice these can also be applied to external reference databases as well)). 

In previous studiees a hierarchy of the reliability of each tool has been applied. And ultimately, concordance of multiple methods is ideal. However, in our experience with complex metagenomics data sets, results from each of these approaches are often contradictory, making interpretation difficult, and much work remains to be done in progressing this field. 

With these caveats in mind, examples are provided below for several of these approaches (CRISPR spacer matching; viral contigs binned in MAGs; tRNA pairwise blase; genome nucleotide identity), together with some helper scripts to generate easily readable summary tables.


***

## CRISPR spacer matching

This method involves: 

- identifying and extracting CRISPR spacer sequences in the data set of filtered and trimmed sequencing reads (prior to assembly)
- pairwise *blast* searches are then run between: 
  - spacer sequences vs viral genomes
  - spacer sequences vs a set of prokaryote metagenome-assembled genomes. 
- These results can then be compiled to identify spacer sequences that match between specific viral genomes and specific MAGs.



NOTE: 

- A limitation of this approach is the extent to which the diversity contained within the MAGs data set reflects the full diversity of the system. Often, MAGs are a simplified set of the reality of the system. As such, we might expect to miss a lot of matches due to gaps in the available data.
- As some amount of micro-diversity is likely to be lost in the process of dereplicating MAGs (e.g. *dRep* applied to MAGs from multiple assemblies), it may be preferable to apply this step to the set of MAGs *prior* to this dereplication. For example, having refined and reduced MAG sets for each assembly (e.g. via DASTool dereplication of MAGs generated by multiple tools for each assembly, followed by manual curation), but prior to a final dereplication across all assemblies via *dRep*. This is more likely to maximise the microdiversity in the MAG set (which may include some of the sites associated with CRISPR spacers) and may result in an increased number of hits for spacer matches. This will also retain useful information on the specific assemblies (samples, in this case) that particular virus-host CRISPR matches were identified in, and whether they were identified across multiple sites (which may strengthen confidence in the prediction).  



#### Data prep 

Concatenate prokaryote MAGs, and also copy over read files

NOTE: 

- In some cases it is necessary to filter out problematic reads that cause *crass* to crash (the output names the reads that are the issue). To enable modifying the reads files without affecting the original set, copy over read files first, then filter, then (re)run *crass*. 

In [ ]:
# working directory
cd /working/dir

# Concatenate bins into all.hosts.fna
mkdir -p 5.host_prediction/crispr/bins
cat /path/to/bin_files/*.fa > 5.host_prediction/crispr/bins/all.hosts.fna

# Copy over reads files
mkdir -p 5.host_prediction/crispr/infiles
cp /path/to/wgs/1.Qual_filtered_trimmomatic/*.fastq 5.host_prediction/crispr/infiles/



If required: remove problematic reads

After a first pass of *crass* (see below), if any runs crashed due to problematic reads these can be filtered out here and then *crass* re-run.

Example where multiple reads from S8 and S9 failed: 

- `sed` here identifies the header row based on ID and deletes this row and the 3 that follow (since sequences cover four lines in *fastq* file format). 
- Note that the read IDs for R1 and R2 will be identical except for the `1` or `2` at the start of the section after the space)

In [ ]:
#!/bin/bash
#SBATCH -A your_project_account
#SBATCH -J 7_crispr_prep
#SBATCH --time 02:00:00
#SBATCH --mem 100MB
#SBATCH --ntasks 1
#SBATCH --cpus-per-task 2
#SBATCH -e 7_crispr_prep.err
#SBATCH -o 7_crispr_prep.out

# working directory
cd /working/dir/5.host_prediction/crispr/infiles

## Remove problematic reads

# S8_R1 problem read 1
sed -i '/^@7001326F:173:H27CLBCX3:1:2114:12778:70365 1:N:0:ATTCAGAA+GTACTGAC/,+3d' S8_R1.fastq
sed -i '/^@7001326F:173:H27CLBCX3:1:2114:12778:70365 2:N:0:ATTCAGAA+GTACTGAC/,+3d' S8_R2.fastq

# S8_R1 problem read 2
sed -i '/^@7001326F:172:H27C7BCX3:1:1211:9175:92520 1:N:0:ATTCAGAA+GTACTGAC/,+3d' S8_R1.fastq
sed -i '/^@7001326F:172:H27C7BCX3:1:1211:9175:92520 2:N:0:ATTCAGAA+GTACTGAC/,+3d' S8_R2.fastq

# S8_R1 problem read 3
sed -i '/^@7001326F:173:H27CLBCX3:1:2213:15674:2212 1:N:0:ATTCAGAA+GTACTGAC/,+3d' S8_R1.fastq
sed -i '/^@7001326F:173:H27CLBCX3:1:2213:15674:2212 2:N:0:ATTCAGAA+GTACTGAC/,+3d' S8_R2.fastq


#### Extract crispr spacers

1. Identify CRISPR spacer sequences from filtered sequencing reads via *crass*
1. get stats via `crisprtools stat` (part of *crass* install)
1. Extract spacer sequences
1. Add sample info into headers of spacer sequences

NOTE:

- *crass* is not currently available as a NeSI module. You will need to download this first.

In [ ]:
#!/bin/bash
#SBATCH -A your_project_account
#SBATCH -J 7_crispr_crass
#SBATCH --time 06:00:00
#SBATCH --mem 3GB
#SBATCH --ntasks 1
#SBATCH --array=1-9
#SBATCH --cpus-per-task 2
#SBATCH -e 7_crispr_crass_%a.err
#SBATCH -o 7_crispr_crass_%a.out

# working directory
cd /working/dir/5.host_prediction

# Paths
crass_path=/nesi/project/uoa02469/Software/crass/bin

# run crass
mkdir -p crispr/S${SLURM_ARRAY_TASK_ID}

${crass_path}/crass \
crispr/infiles/S${SLURM_ARRAY_TASK_ID}_R1.fastq \
crispr/infiles/S${SLURM_ARRAY_TASK_ID}_R2.fastq \
-o crispr/S${SLURM_ARRAY_TASK_ID}/

# Get stats via crisprtools stats
mkdir -p crispr/stats_out

${crass_path}/crisprtools stat \
-ap crispr/S${SLURM_ARRAY_TASK_ID}/crass.crispr \
> crispr/stats_out/S${SLURM_ARRAY_TASK_ID}_crass_stats.txt

# Extract spacer sequences
mkdir -p crispr/spacer_seqs

${crass_path}/crisprtools extract \
-o crispr/ -O crispr/S${SLURM_ARRAY_TASK_ID}/ \
-s crispr/S${SLURM_ARRAY_TASK_ID}/crass.crispr \
> crispr/spacer_seqs/S${SLURM_ARRAY_TASK_ID}_spacers.fa

# Add sample info to sequence headers of spacers.fa files
sed -i "s/>/>S${SLURM_ARRAY_TASK_ID}_/g" crispr/spacer_seqs/S${SLURM_ARRAY_TASK_ID}_spacers.fa


#### *Blastn*: spacers vs viral contigs and MAGs

Run blast comparison between spacers and vOTUs and bins

NOTE: The associated helper script to generate summary tables *requires* that `-outfmt` be set exactly as below.

In [ ]:
# working directory
cd /working/dir/5.host_prediction/
mkdir -p crispr/spacers_blastn

# load blast
module purge
module load BLAST/2.9.0-gimkl-2018b

## Concatenate spacers together
cat crispr/spacer_seqs/*.fa > crispr/spacer_seqs/all_spacer_seqs.fna

## Build index
makeblastdb -in crispr/spacer_seqs/all_spacer_seqs.fna -dbtype nucl \
-out crispr/spacer_seqs/all_spacer_seqs.fna

## Bins
# working directory
cd /working/dir/5.host_prediction/
# Run blastn vs bins
blastn -num_threads 6 \
-query crispr/bins/all.hosts.fna \
-db crispr/spacer_seqs/all_spacer_seqs.fna \
-outfmt "6 qseqid qlen sseqid slen pident length mismatch gapopen qstart qend sstart send evalue bitscore qcovs" \
-out crispr/spacers_blastn/blastn_crisprSpacers.Bins.txt

## vOTUs
# working directory
cd /working/dir/5.host_prediction/
# Run blastn vs viral contigs
blastn -num_threads 6 \
-query ../6.checkv_vOTUs/vOTUs.checkv_filtered.fna \
-db crispr/spacer_seqs/all_spacer_seqs.fna \
-outfmt "6 qseqid qlen sseqid slen pident length mismatch gapopen qstart qend sstart send evalue bitscore qcovs" \
-out crispr/spacers_blastn/blastn_crisprSpacers.vOTUs.txt


#### Compile crispr spacers summary results table

NOTE: 

- `n_hits_threshold` at the beginning sets the number of top matches to keep for each blast search
- The script below *requires* that `-outfmt` be set exactly as above when running the blast searches.
- This script includes filtering to keep only blast matches with ≤ 1 mismatch over the full length of the spacer sequence
- This script also includes adding gtdb predicted MAG taxonomy for each of the matching bins (if this is unavailable, it is useful to run this first, or modify the script below to remove gtdb-related steps)
- This will ultimately be put into a script for ease of use. But for now we can use the python code below

In [ ]:
# working directory
cd /working/dir/5.host_prediction
mkdir -p summary_tables

# Load python
module purge
module load Python/3.8.2-gimkl-2020a
python3

### Import required libraries
import pandas as pd
import numpy as np
import re
import glob

## number of top hits to keep
n_hits_threshold = 3

## File paths
gtdb_taxonomy_path = '/path/to/gtdb/output'
crispr_results_bins = 'crispr/spacers_blastn/blastn_crisprSpacers.Bins.txt'
crispr_results_votus = 'crispr/spacers_blastn/blastn_crisprSpacers.vOTUs.txt'

## GTDB
# Read in gtdb taxonomy to append to each
gtdb_df = pd.concat([pd.read_csv(f, sep='\t') for f in glob.glob("/path/to/gtdb/output/*.summary.tsv")],
                      ignore_index=True)[['user_genome', 'classification']]
gtdb_df.columns = ['crispr_blast_binID', 'crispr_blast_bin_taxonomy_gtdb']

## Bins results
# Read in blast results
bins_df = pd.read_csv(crispr_results_bins, sep='\t', header=None)
# Rename columns
bins_df.columns = ['bin_contig_ID', 'query_length', 'spacer_ID', 'spacer_len', 'pident', 'match_length', 'mismatch', 'gapopen', 'query_start', 'query_end', 'spacer_start', 'spacer_end', 'evalue', 'bitscore', 'query_covs']
# Filter to only keep matches with ≤ 1 mismatch over the full length of the spacer sequence
bins_df = bins_df[(bins_df['spacer_len'] == bins_df['match_length']) & (bins_df['mismatch'] <= 1)]

## vOTUs
# Read in blast results
df = pd.read_csv(crispr_results_votus, sep='\t', header=None)
# Rename columns
df.columns = ['virID', 'query_length', 'spacer_ID', 'spacer_len', 'pident', 'match_length', 'mismatch', 'gapopen', 'query_start', 'query_end', 'spacer_start', 'spacer_end', 'evalue', 'bitscore', 'query_covs']
# Filter to only keep matches with ≤ 1 mismatch over the full length of the spacer sequence
df = df[(df['spacer_len'] == df['match_length']) & (df['mismatch'] <= 1)]
# join with bins results (by spacer_ID) and filter to keep only rows that have hits for both a viral contig and a binned contig
df = pd.merge(df, bins_df, how="left", on="spacer_ID", suffixes=("_vir", "_bin"))
df = df[df['bin_contig_ID'].notnull()].sort_values(by=['virID'])
# Add gtdb taxonomy for bins
df['crispr_blast_binID'] = df['bin_contig_ID'].str.replace(r'_NODE.*', '')
df = pd.merge(df, gtdb_df, how="left", on="crispr_blast_binID").reset_index(drop=True)
# Filter to keep columns of interest
df = df[['virID', 'pident_vir', 'evalue_vir', 'bitscore_vir', 'pident_bin', 'evalue_bin', 'bitscore_bin', 'crispr_blast_binID', 'crispr_blast_bin_taxonomy_gtdb']]
df.columns = ['virID', 'crispr_blast_pident_vir', 'crispr_blast_evalue_vir', 'crispr_blast_bitscore_vir', 'crispr_blast_pident_bin', 'crispr_blast_evalue_bin', 'crispr_blast_bitscore_bin', 'crispr_blast_binID', 'crispr_blast_bin_taxonomy_gtdb']

## filter to only keep top n hits for each vOTU_ID
# ERROR handling: If n_hits_threshold greater than or equal to max counts, need to modify n_hits_threshold
MAX_VALUE_COUNTS = df.groupby('virID')['virID'].value_counts().max()
if n_hits_threshold >= MAX_VALUE_COUNTS:
    n_hits_threshold_edit = MAX_VALUE_COUNTS-1
else:
    n_hits_threshold_edit = n_hits_threshold

# filter by n_hits_threshold
df = df[df.index.isin(df.groupby('virID')['crispr_blast_evalue_vir'].nsmallest(n_hits_threshold_edit).index.get_level_values(1))].sort_values(by=['virID', 'crispr_blast_evalue_vir'])
# pivot wider and apply suffix to multiple hits
df['idx'] = '_'+(df.groupby(['virID']).cumcount() + 1).astype(str)
df = (
    df.pivot_table(
        index=['virID'], 
        columns=['idx'], 
        values=['crispr_blast_pident_vir', 'crispr_blast_evalue_vir', 'crispr_blast_bitscore_vir', 'crispr_blast_pident_bin', 'crispr_blast_evalue_bin', 'crispr_blast_bitscore_bin', 'crispr_blast_binID', 'crispr_blast_bin_taxonomy_gtdb'], 
        aggfunc='first'
    )
)
df.columns = [''.join(col) for col in df.columns]
df = df.reset_index()

## Write out summary table
df.to_csv('summary_tables/summary_table.crisprSpacers_blastn.tsv', sep='\t', index=False)

quit()


***

## Binned viral contigs

- Identify any viral contigs that were co-binned when generating the Prokaryote MAGs data set.
- Note that this requires a bin2contig_lookupTable matching contigIDs of all contigs binned into MAGs against their MAG (bin) ID (two columns, with headers `contigID` and `binID`). See **VirusStudies_WGS_5_Read_mapping** for more information and some python code to generate this table.
- The `vOTUs_lookupTable.txt` was originally generated during dereplication of viral contigs into vOTUs back in **VirusStudies_WGS_3_Viral_Identification**.
- *Optional*: The script below also incorporates MAG taxonomy (gtdb results) and checkM, as these are useful metrics to include here.
- This will ultimately be put into a script for ease of use. But for now we can use the python code below.

In [ ]:
# working directory
cd /working/dir
mkdir -p 5.host_prediction/binned_contigs/
mkdir -p 5.host_prediction/summary_tables

# Load python
module purge
module load Python/3.8.2-gimkl-2020a
python3

### Import required libraries
import pandas as pd
import numpy as np
import re
from Bio.SeqIO.FastaIO import SimpleFastaParser
import os

# File paths
votus_lookupTable_path = '1.viral_identification/5.cluster_vOTUs/vOTUs_lookupTable.txt'
viral_contigs_path = '1.viral_identification/6.checkv_vOTUs/vOTUs.checkv_filtered.fna'
MAGs_bin2contig_lookupTable_path = '/path/to/MAGs_bin2contig_lookupTable.tsv'
MAGs_checkM_summary_path = '/path/to/checkm_bin_summary.txt'
MAGS_gtdb_output_path = '/path/to/gtdb/output'

# vOTUs
votus_dict = {}
with open(viral_contigs_path, 'r') as read_fasta:
    for name, seq in SimpleFastaParser(read_fasta):
        votus_dict[name] = seq

# Convert to dataframe
votu_df = pd.DataFrame.from_dict(votus_dict.items())
votu_df.columns = ['vOTU_ID_full', 'seq']
votu_df['vOTU_ID'] = votu_df['vOTU_ID_full'].str.replace(r'(vOTU_\d+).*', r'\1', regex=True)

# Join with lookuptable to get original contigIDs
lookupTable_df = pd.read_csv(votus_lookupTable_path, sep='\t')
# Trim contigID back to orignal form
lookupTable_df['contigID'] = lookupTable_df['cluster_rep_contigID'].str.replace(r'(cov_.*\.\d+).*', r'\1', regex=True)
votu_df = pd.merge(votu_df, lookupTable_df, left_on="vOTU_ID", right_on="vOTU", how="left")

# join with MAGs bin2contig lookupTable to identify any matches between the datasets
mag_bin2contig_df = pd.read_csv(MAGs_bin2contig_lookupTable_path, sep='\t')
votu_df = pd.merge(votu_df, mag_bin2contig_df, on="contigID", how="left")

# binned vOTU contigs (exclude any rows with no matches between vOTUs and MAGs)
binned_votus_df = votu_df[~votu_df['binID'].isna()]

# For reference: Counts of binIDs with vOTU matches (Open to view)
vir2bin_matches_df = binned_votus_df['binID'].value_counts().rename_axis('bins').reset_index(name='counts')

# Write out results table
binned_votus_df = binned_votus_df[['vOTU_ID_full', 'binID']]
binned_votus_df.columns = ['virID', 'cobinned_binID']
binned_votus_df.to_csv('5.host_prediction/binned_contigs/vOTUs.cobinned.tsv', sep='\t', index=False)

# Optional: read in MAG taxonomy (gtdb) and checkM and join with cobinned results
gtdb_df = pd.concat([pd.read_csv(f, sep='\t') for f in glob.glob(MAGS_gtdb_output_path+"/*.summary.tsv")],
                      ignore_index=True)[['user_genome', 'classification']]
gtdb_df.columns = ['binID', 'cobinned_bin_taxonomy_gtdb']
checkm_df = pd.read_csv(MAGs_checkM_summary_path, sep='\t')[['Bin Id', 'Completeness', 'Contamination', 'Strain heterogeneity']]
checkm_df.columns = ['binID', 'cobinned_checkm_Completeness', 'cobinned_checkm_Contamination', 'cobinned_checkm_Strain heterogeneity']
mag_stats_df = pd.merge(gtdb_df, checkm_df, how="outer", on="binID").reset_index(drop=True).rename(columns={"binID": "cobinned_binID"})
# Join MAG stats with cobinned results
full_df = pd.merge(binned_votus_df, mag_stats_df, how="left", left_on="cobinned_binID", right_on="cobinned_binID").reset_index(drop=True)

# Write out summary table
full_df.to_csv('5.host_prediction/summary_tables/summary_table.vir_contigs.cobinned.tsv', sep='\t', index=False)

quit()


***

## tRNA pairwise *blast*

Pairwise blast comparison of tRNA sequences predicted from bin (MAG) files and viral contigs

NOTE:

- (As with CRISPR matching above), as some amount of micro-diversity is likely to be lost in the process of dereplicating MAGs (e.g. *dRep* applied to MAGs from multiple assemblies), it may be preferable to apply this step to the set of MAGs *prior* to this dereplication. This is more likely to maximise the microdiversity in the MAG set (and potentailly the diversity of tRNA) and may result in an increased number of hits for tRNA matches. This will also retain useful information on the specific assemblies (samples, in this case) that particular tRNA matches were identified in, and whether they were identified across multiple sites (which may strengthen confidence in the prediction).
- *aragorn* is not currently available as a NeSI module. You will need to download and install this first.


#### Set up working directory and copy infiles over

NOTE:

- In the next step, we will predict tRNA sequences for each individual "genome" via *aragorn*. Here we will first generate individual files for each "genome" (contig) in the vOTUs data set. A simple python script to achieve this is given below.
- If your bin (MAG) file extensions are something other than `.fa` (e.g. `.fna`), ammend the script below in both the bin files copy step *and* the step writing out the individual viral contig files (so that all extensions match for ease of use downstream), and *also* in the subsequent *aragorn* step below.

In [ ]:
cd /working/dir

# Make output directories
mkdir -p 5.host_prediction/tRNA_blast/infiles

## bin files 
cp /path/to/bin_files/*.fa 5.host_prediction/tRNA_blast/infiles/

## Viral contigs
# Split virus fasta file into individual sequences
# Load python
module purge
module load Python/3.8.2-gimkl-2020a
python3
# Import required libraries
import pandas as pd
import numpy as np
from Bio.SeqIO.FastaIO import SimpleFastaParser
# Write individual fasta files
with open('1.viral_identification/6.checkv_vOTUs/vOTUs.checkv_filtered.fna', 'r') as read_fasta:
    for name, seq in SimpleFastaParser(read_fasta):
        with open('5.host_prediction/tRNA_blast/infiles/'+name+'.fa', 'w') as write_fasta:
            write_fasta.write(">" + str(name) + "\n" + str(seq) + "\n")

quit()


#### Identify tRNA

Run *aragorn* on each "genome" and prep for pairwise *blast*

Note: 

- The prep steps include: Removing any empty files (i.e. those with no predicted tRNAs); Modifying contig headers in aragorn output files (to simplify downstream use); Concatenating viral tRNAs and host tRNAs in two files.
- The concatenation step for tRNA files relating to MAGs is currently written based on naming generated by metabat, maxbin, and concoct. If this does not match your MAG file names, ammend this line in the code below (e.g. if your MAG files are all simply labelled "MAG_n.fna" then  `cat MAG_*.trna.fna >  concatenated_tRNA/host_tRNA.fna` will suffice)
- *aragorn* is not currently available as a NeSI module. You will need to download and install this first.

In [ ]:
#!/bin/bash -e
#SBATCH -A your_project_account
#SBATCH -J 7_tRNA_aragorn
#SBATCH --time 03:00:00
#SBATCH --mem=1GB
#SBATCH --ntasks=1
#SBATCH --cpus-per-task=12
#SBATCH -e 7_tRNA_aragorn.err
#SBATCH -o 7_tRNA_aragorn.out
#SBATCH --profile=task

# load dependencies
module purge

# Working dir
mkdir -p /working/dir/5.host_prediction/tRNA_blast/aragorn_out
cd /working/dir/5.host_prediction/tRNA_blast

## Run Aragorn
for file in infiles/*.fa; do 
    filename=$(basename ${file} .fa)
    /path/to/Software/aragorn_v1.2.38/aragorn \
    -t -gcstd -l -a -q -rn -fon \
    -o aragorn_out/${filename}.aragorn \
    ${file}
done

## Prep for pairwise blast
cd aragorn_out/

# Remove empty files
find . -type f -empty -delete
# modify contig headers
for file in *aragorn; do 
    filename=$(basename ${file} .aragorn)
    sed -e "s/>/>${filename}_/g" -e 's/\(.*\) .*/\1/' -e 's/[^a-zA-Z0-9>]/_/g' -e 's/_$//g' ${file} > ${file}.trna.fna
done

# concatenate into viral_tRNA.fna and host_tRNA.fna
mkdir -p concatenated_tRNA
cat vOTU*.trna.fna > concatenated_tRNA/viral_tRNA.fna
cat *metabat*.trna.fna *maxbin*.trna.fna *concoct*.trna.fna > concatenated_tRNA/host_tRNA.fna

# Compile tRNA with aragorn headers in full (i.e. the original aragorn output files, rather than those with modified contig headers)
mkdir -p concatenated_tRNA_aragorn
cat vOTU*.aragorn > concatenated_tRNA_aragorn/viral_tRNA.fna
cat *metabat*.aragorn *maxbin*.aragorn *concoct*.aragorn > concatenated_tRNA_aragorn/host_tRNA.fna



#### Pairwise blast of tRNA sequences

NOTE:

- `-outfmt` must be exactly as below for the subsequent script that generates the summary table.


In [ ]:
# load module(s)
module purge
module load BLAST/2.9.0-gimkl-2018b

# Working dir
cd /working/dir/5.host_prediction/tRNA_blast

# blast db of host_tRNA.fna
makeblastdb -in aragorn_out/concatenated_tRNA/host_tRNA.fna -dbtype nucl \
-out aragorn_out/concatenated_tRNA/host_tRNA.fna

# Pairwise blast of viral_tRNA v. host_tRNA
blastn \
-query aragorn_out/concatenated_tRNA/viral_tRNA.fna \
-db aragorn_out/concatenated_tRNA/host_tRNA.fna \
-num_threads 8 \
-outfmt "6 qseqid qlen sseqid slen pident length mismatch gapopen qstart qend sstart send evalue bitscore qcovs" \
-num_alignments 5 \
-perc_identity 90 -dust no \
-out vOTUs.tRNA_blast_virus_host.txt


#### Compile tRNA blast filtered summary results table

NOTE: 

- `n_hits_threshold` at the beginning sets the number of top matches to keep for each blast search
- The script below *requires* that `-outfmt` be set exactly as above when running the blast search.
- This script includes a filtering step to keep only blast matches with ≥ 90% nucleotide identity over ≥ 90% length of sequence
- This script also includes adding gtdb predicted MAG taxonomy for each of the matching bins (if this is unavailable, it is useful to run this first, or modify the script below to remove gtdb-related steps)
- This will ultimately be put into a script for ease of use. But for now we can use the python code below.

In [ ]:
# working directory
cd /working/dir/5.host_prediction
mkdir -p summary_tables

# Load python
module purge
module load Python/3.8.2-gimkl-2020a
python3

### Import required libraries
import pandas as pd
import numpy as np
import re
import glob

### keep n hit threshold setting
n_hits_threshold = 3

## File paths
gtdb_taxonomy_path = '/path/to/gtdb/output'
trna_blast_path = 'tRNA_blast/vOTUs.tRNA_blast_virus_host.txt'

## Read in gtdb taxonomy to append to each
gtdb_df = pd.concat([pd.read_csv(f, sep='\t') for f in glob.glob(gtdb_taxonomy_path+"/*.summary.tsv")],
                      ignore_index=True)[['user_genome', 'classification']]
gtdb_df.columns = ['tRNA_blast_binID', 'tRNA_blast_bin_taxonomy_gtdb']

## Read in and process blast results
df = pd.read_csv(trna_blast_path', sep='\t', header=None)
# Rename columns
df.columns = ['vir_tRNA_ID', 'vir_tRNA_length', 'bin_tRNA_ID', 'bin_tRNA_len', 'pident', 'match_length', 'mismatch', 'gapopen', 'vir_tRNA_start', 'vir_tRNA_end', 'bin_tRNA_start', 'bin_tRNA_end', 'evalue', 'bitscore', 'vir_covs']
# Add virID column
df['virID'] = df['vir_tRNA_ID'].str.replace(r'(vOTU_.*)_.*_.*_tRNA.*', r'\1')
# Add % length of sequence match
df['match_length_pct'] = ((df['match_length'] / df[["vir_tRNA_length", "bin_tRNA_len"]].min(axis=1))*100).round(2)
# Filter to only keep matches with ≥ 90% nucleotide identity and ≥ 90% length of (shortest) tRNA sequence
df = df[(df['pident'] >= 90) & (df['match_length_pct'] >= 90)]
# Add gtdb taxonomy for bins
df['tRNA_blast_binID'] = df['bin_tRNA_ID'].str.replace(r'(contigs).*', r'\1').str.replace(r'_', r'.').str.replace(r'.sub', r'_sub')
df = pd.merge(df, gtdb_df, how="left", on="tRNA_blast_binID").reset_index(drop=True)
# Filter to keep columns of interest
df = df[['virID', 'tRNA_blast_binID', 'tRNA_blast_bin_taxonomy_gtdb', 'pident', 'evalue', 'bitscore']]
df.columns = ['virID', 'tRNA_blast_binID', 'tRNA_blast_bin_taxonomy_gtdb', 'tRNA_blast_pident', 'tRNA_blast_evalue', 'tRNA_blast_bitscore']

## filter to only keep top n hits for each vOTU_ID
# ERROR handling: If n_hits_threshold greater than or equal to max counts, need to modify n_hits_threshold
MAX_VALUE_COUNTS = df.groupby('virID')['virID'].value_counts().max()
if n_hits_threshold >= MAX_VALUE_COUNTS:
    n_hits_threshold_edit = MAX_VALUE_COUNTS-1
else:
    n_hits_threshold_edit = n_hits_threshold

# Filter based on n_hits_threshold
df = df[df.index.isin(df.groupby('virID')['tRNA_blast_evalue'].nsmallest(n_hits_threshold_edit).index.get_level_values(1))].sort_values(by=['virID', 'tRNA_blast_evalue'])
# pivot wider and apply suffix to multiple hits
df['idx'] = '_'+(df.groupby(['virID']).cumcount() + 1).astype(str)
df = (df.pivot_table(index=['virID'], 
                               columns=['idx'], 
                               values=['tRNA_blast_binID', 'tRNA_blast_bin_taxonomy_gtdb', 'tRNA_blast_pident', 'tRNA_blast_evalue', 'tRNA_blast_bitscore'], 
                               aggfunc='first'))
df.columns = [''.join(col) for col in df.columns]
df = df.reset_index()

## Write out summary table
df.to_csv('summary_tables/summary_table.tRNA_blast_virus_host.tsv', sep='\t', index=False)

quit()


***

## Whole genome nucleotide sequence identity

Run pairwise *blast* searches between vOTUs and MAGs to assess whole genome sequence identity

NOTE:

- (As with CRISPR spacer and tRNA matching above), as some amount of micro-diversity is likely to be lost in the process of dereplicating MAGs (e.g. *dRep* applied to MAGs from multiple assemblies), it may be preferable to apply this step to the set of MAGs *prior* to this dereplication. This is more likely to maximise the microdiversity in the MAG set (and potentailly the diversity of tRNA) and may result in an increased number of hits for tRNA matches. This will also retain useful information on the specific assemblies (samples, in this case) that particular tRNA matches were identified in, and whether they were identified across multiple sites (which may strengthen confidence in the prediction).
- `-outfmt` must be exactly as below for the subsequent script that generates the summary table.


#### Prep infiles and run *blast*

In [ ]:
# Working directory
cd /working/dir

# Concatenate bins into all.hosts.fna
mkdir -p 5.host_prediction/pairwise_blast/infiles
cat /path/to/bin_files/*.fa > 5.host_prediction/pairwise_blast/infiles/all.hosts.fna

# load blast
module purge
module load BLAST/2.9.0-gimkl-2018b

# Generate database from all.hosts.fna
makeblastdb \
-in 5.host_prediction/pairwise_blast/infiles/all.hosts.fna -dbtype nucl \
-out 5.host_prediction/pairwise_blast/infiles/all.hosts.fna

# Pairwise blast against vOTUs (blast pairwise comparisons based on nucleotide sequence identity)
blastn -num_threads 10 \
-query 1.viral_identification/6.checkv_vOTUs/vOTUs.checkv_filtered.fna \
-db 5.host_prediction/pairwise_blast/infiles/all.hosts.fna \
-outfmt "6 qseqid qlen sseqid slen pident length mismatch gapopen qstart qend sstart send evalue bitscore qcovs" \
-perc_identity 70 \
-out 5.host_prediction/pairwise_blast/vOTUs.pairwise_blast.txt


#### Compile filtered summary results tables

NOTE:

- `n_hits_threshold` at the beginning sets the number of top matches to keep for each blast search
- The script below *requires* that `-outfmt` be set exactly as above when running the blast search.
- This script includes a filtering step to keep only blast matches with: e-value ≤ 0.001; nucleotide identity ≥ 70%; bit-score ≥ 50
- This script also includes adding gtdb predicted MAG taxonomy for each of the matching bins (if this is unavailable, it is useful to run this first, or modify the script below to remove gtdb-related steps)
- This will ultimately be put into a script for ease of use. But for now we can use the python code below.

In [ ]:
# working directory
cd /working/dir/5.host_prediction
mkdir -p summary_tables

# Load python
module purge
module load Python/3.8.2-gimkl-2020a
python3

### Import required libraries
import pandas as pd
import numpy as np
import re
import glob

### number of top hits to keep
n_hits_threshold = 3

## File paths
gtdb_taxonomy_path = '/path/to/gtdb/output'
nucleotide_identity_blast_path = 'pairwise_blast/vOTUs.pairwise_blast.txt'

### Read in gtdb taxonomy to append to each
gtdb_df = pd.concat([pd.read_csv(f, sep='\t') for f in glob.glob(gtdb_taxonomy_path+"/*.summary.tsv")],
                      ignore_index=True)[['user_genome', 'classification']]
gtdb_df.columns = ['pairwise_blast_binID', 'pairwise_blast_bin_taxonomy_gtdb']

## Read in and process blast results
df = pd.read_csv(nucleotide_identity_blast_path, sep='\t', header=None)
# Rename columns
df.columns = ['virID', 'vir_length', 'binID', 'bin_len', 'pident', 'match_length', 'mismatch', 'gapopen', 'vir_start', 'vir_end', 'bin_start', 'bin_end', 'evalue', 'bitscore', 'vir_covs']
# Filter to only keep matches with: e-value ≤ 0.001 & nucleotide identity ≥ 70% & bit-score ≥ 50 
df = df[(df['pident'] >= 70) & (df['evalue'] <= 0.001) & (df['bitscore'] >= 50)]
# Add gtdb taxonomy for bins
df['pairwise_blast_binID'] = df['binID'].str.replace(r'(contigs).*', r'\1').str.replace(r'_', r'.').str.replace(r'.sub', r'_sub')
df = pd.merge(df, gtdb_df, how="left", on="pairwise_blast_binID").reset_index(drop=True)
# Filter to keep columns of interest
df = df[['virID', 'pairwise_blast_binID', 'pairwise_blast_bin_taxonomy_gtdb', 'pident', 'evalue', 'bitscore']]
df.columns = ['virID', 'pairwise_blast_binID', 'pairwise_blast_bin_taxonomy_gtdb', 'pairwise_blast_pident', 'pairwise_blast_evalue', 'pairwise_blast_bitscore']

## filter to only keep top n hits for each vOTU_ID
# ERROR handling: If n_hits_threshold greater than or equal to max counts, need to modify n_hits_threshold
MAX_VALUE_COUNTS = df.groupby('virID')['virID'].value_counts().max()
if n_hits_threshold >= MAX_VALUE_COUNTS:
    n_hits_threshold_edit = MAX_VALUE_COUNTS-1
else:
    n_hits_threshold_edit = n_hits_threshold

# Filter based on n_hits_threshold
df = df[df.index.isin(df.groupby('virID')['pairwise_blast_evalue'].nsmallest(n_hits_threshold_edit).index.get_level_values(1))].sort_values(by=['virID', 'pairwise_blast_evalue'])
# pivot wider and apply suffix to multiple hits
df['idx'] = '_'+(df.groupby(['virID']).cumcount() + 1).astype(str)
df = (df.pivot_table(index=['virID'], 
                               columns=['idx'], 
                               values=['pairwise_blast_binID', 'pairwise_blast_bin_taxonomy_gtdb', 'pairwise_blast_pident', 'pairwise_blast_evalue', 'pairwise_blast_bitscore'], 
                               aggfunc='first'))
df.columns = [''.join(col) for col in df.columns]
df = df.reset_index()

## Write out summary table
df.to_csv('summary_tables/summary_table.pairwise_blast.tsv', sep='\t', index=False)

quit()


***

## Host prediction full summary table

Generate a summary table combining all host prediction results.

NOTE:

- Output files to compile:
  - crispr spacers: `5.host_prediction/summary_tables/summary_table.crisprSpacers_blastn.tsv`
  - binned viral contigs: `5.host_prediction/summary_tables/summary_table.vir_contigs.cobinned.tsv`
  - tRNA blast: `5.host_prediction/summary_tables/summary_table.tRNA_blast_virus_host.tsv`
  - pairwise blast: `5.host_prediction/summary_tables/summary_table.pairwise_blast.tsv`
- The script below also adds vOTU *checkV* stats, as this can be useful when assessing predicted host matches
- This will ultimately be put into a script for ease of use. But for now we can use the python code below.


In [ ]:
# working directory
cd /working/dir

# Load python
module purge
module load Python/3.8.2-gimkl-2020a
python3

### Import required libraries
import pandas as pd
import numpy as np
import re

## Import all results tables
# crispr blast
crispr_df = pd.read_csv('5.host_prediction/summary_tables/summary_table.crisprSpacers_blastn.tsv', sep='\t')
# binned viral contigs
cobinned_df = pd.read_csv('5.host_prediction/summary_tables/summary_table.vir_contigs.cobinned.tsv', sep='\t')
# tRNA blast
tRNA_df = pd.read_csv('5.host_prediction/summary_tables/summary_table.tRNA_blast_virus_host.tsv', sep='\t')
# pairwise blast
pairwise_df = pd.read_csv('5.host_prediction/summary_tables/summary_table.pairwise_blast.tsv', sep='\t')
# checkV results
checkv_df = pd.read_csv('1.viral_identification/6.checkv_vOTUs/vOTUs.checkv_filtered_quality_summary.tsv', sep='\t', ignore_index=True).drop(columns=['Unnamed: 0', 'provirus', 'proviral_length']).add_prefix('checkv_')

## Join all results tables together into summary table
summary_df = pd.merge(
    tRNA_df, how="outer", on=['virID', 'dataset']).merge(
    pairwise_df, how="outer", on=['virID', 'dataset']).merge(
    crispr_df, how="outer", on=['virID', 'dataset']).merge(
    cobinned_df, how="outer", on=['virID', 'dataset']).sort_values(by=['dataset', 'virID']).reset_index(drop=True)

# Trim contigID to match checkv's output (trim off the '__checkv_excised...' bits that we added earlier)
summary_df['contig_id'] = summary_df['virID'].str.replace(r'_\d+__checkv_excised.*', r'', regex=True)

# Merge checkv results onto the summary table
summary_df = pd.merge(summary_df, checkv_df, how="left", left_on='contig_id', right_on='checkv_contig_id').drop(columns=['contig_id'])

## Reorder columns, and write out summary table
summary_df=summary_df[['dataset', 'virID'] + 
                       [col for col in summary_df.columns if 'checkv' in col] + 
                       [col for col in summary_df.columns if 'cobinned' in col] + 
                       [col for col in summary_df.columns if 'crispr' in col] + 
                       [col for col in summary_df.columns if 'tRNA' in col] + 
                       [col for col in summary_df.columns if 'pairwise' in col]]
summary_df.to_csv('5.host_prediction/summary_tables/summary_table.All_host_predictions.tsv', sep='\t', index=False)

quit()


***